In [1]:
import pandas as pd
import re
import time
import binascii

In [2]:
small_dataset = 'data/news_articles_small.csv'
df_small_dataset = pd.read_csv(small_dataset)

In [3]:
#Pre-process data
#Convert to lowercase.
df_small_dataset['article'] = df_small_dataset['article'].str.lower()

#Remove punctuation
p = re.compile(r'[^\w\s]+')
df_small_dataset['article'] = [p.sub('', x) for x in df_small_dataset['article'].tolist()]

In [4]:
"""
Split each document in a list of words

small_dataset_split = [
    [documentID, document_text]
]
"""

small_dataset_split = []
for idx, row in df_small_dataset.iterrows():
    small_dataset_split.append([row[0], row[1].split()])

In [5]:

#Add shingles with ngram 4
#Source: https://github.com/chrisjmccormick/MinHash/blob/master/runMinHashExample.py
shingledDocs = {}
docIds = []

t0 = time.time()

totalShingles = 0
for docId, article in small_dataset_split:
    shingles = set()
    for i in range(0, len(article) - 3):
        shingle = article[i]+ " " + article[i + 1] + " " + article[i + 2] + " " + article[i + 3]

        crc =  binascii.crc32(shingle.encode()) & 0xffffffff
        shingles.add(crc)

    shingledDocs[docId]= shingles
    docIds.append(docId)
    totalShingles = totalShingles + (len(article) - 3)

t1 = time.time()
print('Time spent: ', t1-t0)


Time spent:  0.3011922836303711
